# NPZD slab model

For the first model example I will need:
- Forcing input
- Forcing_fluxes:
    - mixing
    - sinking
- 4 components (single) N, P, Z, D
- Fluxes:
TODO: GO OVER THE equations! Base model structure in math!
    - Grazing - linked to all NPZD
    - PhytoplanktonGrowth
        - Light - Dependence
        - Nutrients - Dependence
        - Temp - Dependence
    - mortality
    - remineralisation
    - higher order pred, etc

In [13]:
import numpy as np
import xsimlab as xs

In [14]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra


from phydra.utility.xsimlabwrappers import phydra_setup

In [15]:
from phydra.processes.main import Grid0D, Boundary0D
from phydra.processes.components import Component, Time, make_Component
from phydra.processes.environments import BaseEnvironment, Slab
from phydra.processes.fluxes import Flux, LimitedGrowth
from phydra.processes.forcingfluxes import Mixing, Sinking, Upwelling, make_FX_flux
from phydra.processes.forcing import SinusoidalMLD, ConstantN0
from phydra.processes.gekkocontext import GekkoContext, GekkoSolve, InheritGekkoContext

In [16]:
from phydra.processes.forcing import NutrientForcing, MLDForcing

In [17]:
print(dir())

['BaseEnvironment', 'Boundary0D', 'Component', 'ConstantN0', 'Flux', 'GekkoContext', 'GekkoSolve', 'Grid0D', 'In', 'InheritGekkoContext', 'LimitedGrowth', 'MLDForcing', 'Mixing', 'NutrientForcing', 'Out', 'Sinking', 'SinusoidalMLD', 'Slab', 'Time', 'Upwelling', '_', '_10', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'exit', 'get_ipython', 'in_ds', 'inspect', 'make_Component', 'make_FX_flux', 'model', 'np', 'os', 'phydra', 'phydra_setup', 'quit', 'sys', 'xs']


In [18]:
print('xarray-simlab version: ', xs.__version__)
print('phydra version: ', phydra.__version__)

xarray-simlab version:  0.4.1
phydra version:  0+untagged.67.g5c2f315.dirty


In [19]:
xs.process_info(make_FX_flux(Sinking, 'S'))

xs.process_info(make_FX_flux(Upwelling, 'U'))

<class 'phydra.processes.forcingfluxes.Sinking'> S
<class 'phydra.processes.forcingfluxes.Sinking_S'>
<Sinking_S  (xsimlab process)>
Variables:
    m               [in] <--- GekkoContext.m
    gk_context      [in] <--- GekkoContext.context
    gk_SVs          [in] <--- GekkoContext.SVs
    gk_SVshapes     [in] <--- GekkoContext.SVshapes
    gk_Fluxes       [in] <--- GekkoContext.Fluxes
    gk_Flux_Int     [in] <--- GekkoContext.Flux_Intermediates
    gridshape       [in] <--- GekkoContext.shape
    MLD             [in] <--- Slab.MLD
    MLD_deriv       [in] <--- Slab.MLD_deriv
    N0_forcing      [in] <--- Slab.N0_forcing
    kappa           [in] constant mixing coefficient
    label          [out]
    fx_index       [out] S 
    fx_output      [out] ('S', 'time') 
    C_labels        [in] S label of component(s) that grows
Simulation stages:
    initialize
    finalize_step
<class 'phydra.processes.forcingfluxes.Upwelling'> U
<class 'phydra.processes.forcingfluxes.Upwelling_U'>
<Upwel

In [20]:
model = xs.Model({
    'time':Time, 'grid':Grid0D, 'ibcs':Boundary0D, 
    
    'context':GekkoContext,
                  
    'N':make_Component('Nutrient','N'), 'P':make_Component('Phytoplankton','P'),
    'Z':make_Component('Zooplankton','Z'), 'D':make_Component('Detritus','D'),
    'Env':Slab, 
    
    # Fluxes
    'P_Growth':LimitedGrowth, 
    
    # ForcingFluxes
    'Sinking':make_FX_flux(Sinking, 'S'),
    'Upwelling':make_FX_flux(Upwelling, 'U'),
    
    'MLD':SinusoidalMLD, 
    'N0':ConstantN0, 
    
    'solver':GekkoSolve
})

model

<class 'phydra.processes.forcingfluxes.Sinking'> S
<class 'phydra.processes.forcingfluxes.Sinking_S'>
<class 'phydra.processes.forcingfluxes.Upwelling'> U
<class 'phydra.processes.forcingfluxes.Upwelling_U'>


<xsimlab.Model (15 processes, 21 inputs)>
context
time
    days            [in] ('time',) time in days
grid
    length          [in]
    shape           [in]
ibcs
    ibc             [in] Initial Boundary Conditions
N
    init            [in]
    dim             [in]
P
    dim             [in]
    init            [in]
Z
    init            [in]
    dim             [in]
D
    init            [in]
    dim             [in]
N0
    constant_N0     [in] value of the constant Nutrient forcing
MLD
Env
P_Growth
    R_label         [in] label of ressource component that is consumed
    mu              [in] Maximum growth rate of component
    halfsat         [in] half-saturation constant of nutrient uptak...
    C_label         [in] label of component that grows
Sinking
    C_labels        [in] S label of component(s) that grows
    kappa           [in] constant mixing coefficient
Upwelling
    C_labels        [in] U label of component(s) that grows
    kappa           [in] constant mixing coeff

In [26]:
model.index_vars 


[('time', 'time'),
 ('N', 'index'),
 ('P', 'index'),
 ('Z', 'index'),
 ('D', 'index'),
 ('Env', 'components'),
 ('Sinking', 'fx_index'),
 ('Upwelling', 'fx_index')]

In [22]:
in_ds = phydra_setup(
    model=model,
    input_vars={
        'time__days': ('time', np.arange(0, 365*3)),
        
        'N':{'dim':1,'init':1}, 
        'P':{'dim':1,'init':1},
        'Z':{'dim':1,'init':1},
        'D':{'dim':1,'init':1},
        
        'P_Growth':{'mu':1.,
               'C_label':'P',
               'R_label':'N',
               'halfsat':0.5},
        
        'Sinking':{'kappa':0.1,
               'C_labels':['D','P']},
        
        'Upwelling':{'kappa':0.1,
               'C_labels':['N']},
        
        'N0__constant_N0':2.
    },
    output_vars={
        'Env__outputs':None
    }
                 )

In [23]:
with model:
    out_ds = in_ds.xsimlab.run()

Initializing Gekko Context
Initializing Model Time
Initializing component  N
['N']
Initializing component  P
['P']
Initializing component  Z
['Z']
Initializing component  D
['D']
initializing forcing base now
initializing forcing base now
Initialize NutrientDependency
Growth Dependency_component: P []
GK flux SVFluxes dict stores: dict_items([('P', [0]), ('N', [0])])
Flux label:  Sinking
SVFluxes dict stores: dict_items([('Sinking', [0, 0])])
Flux label:  Upwelling
SVFluxes dict stores: dict_items([('Sinking', [0, 0]), ('Upwelling', [0])])
Assembling equation for component  N
FLUXES: v2 [0, 0]
[0, 0]
Assembling equation for component  P
FLUXES: v3 [0, 0]
[0, 0]
Assembling equation for component  Z
FLUXES: v4 []
[]
Assembling equation for component  D
FLUXES: v5 [0]
[0]
SolveInit
Model context dict stores: dict_items([('shape', {'env': 1}), ('comp_dims', {'N': 1, 'P': 1, 'Z': 1, 'D': 1})])
ModelSolve done
Storing output component  N
Storing output component  P
Storing output component  

ValueError: parameter 'value': expected array with shape (2,), got (1,)

In [24]:
out_ds

NameError: name 'out_ds' is not defined

In [ ]:
out_ds.Env__outputs.set_index(time='time',).plot.line(x='time')